In [138]:
import tushare as ts
import pandas as pd
import sqlite3
from datetime import datetime
from datetime import timedelta
import sys
import time

In [3]:
pro = ts.pro_api('881ddd8e044ca6f5ca1fb55dd720ec177fc1501af9aac377c79ce053')

In [5]:
df = pro.stock_basic(**{
    "ts_code": "",
    "name": "",
    "exchange": "",
    "market": "",
    "is_hs": "",
    "list_status": "",
    "limit": "",
    "offset": ""
}, fields=[
    "ts_code",
    "symbol",
    "name",
    "area",
    "industry",
    "market",
    "list_date",
    "curr_type",
    "exchange",
    "enname",
    "fullname"
])        

In [64]:
conn = sqlite3.connect('../9900-test-db.sqlite')

for i in range(len(df)):
    data = df.iloc[i]
    day = datetime.strptime(data['list_date'], '%Y%m%d').strftime('%Y-%m-%d')

    insert_sql = "INSERT INTO stock_info\
    (ts_code, symbol, name, area, industry, fullname, enname, market, exchange, list_date) VALUES\
    (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);"
    
    insert_data = (
        data['ts_code'], data['symbol'],data['name'],data['area'],
        data['industry'],data['fullname'],data['enname'], data['market'],
        data['exchange'],day
    )
    
    try:
        cur = conn.cursor()
        cur.execute(insert_sql, insert_data)
        conn.commit()
    except sqlite3.IntegrityError:
        pass
    except:
        print("Unexpected error when insert %s:",data.values, sys.exc_info())
        
conn.close()

Opened database successfully


In [161]:
def get_stock_info():
    stock_info= {}
    conn = sqlite3.connect('../9900-test-db.sqlite')
    
    select_sql = "select sid, ts_code from stock_info;"
    cur = conn.cursor()
    cur.execute(select_sql)
    for result in cur:
        stock_info[result[0]] = {"ts_code": result[1]}

    conn.close()
    
    return stock_info

stock_info = get_ts_code_list()

In [162]:
def get_stock_info_by_ts_code():
    stock_info_by_ts_code= {}
    conn = sqlite3.connect('../9900-test-db.sqlite')

    select_sql = "select sid, ts_code from stock_info;"
    cur = conn.cursor()
    cur.execute(select_sql)
    for result in cur:
        stock_info_by_ts_code[result[1]] = {"sid": result[0]}

    conn.close()

    return stock_info_by_ts_code

stock_info_by_ts_code = get_stock_info_by_ts_code()

In [180]:
def save_trade_df(df):
    conn = sqlite3.connect('../9900-test-db.sqlite')

    for i in range(len(df)):
        data = df.iloc[i]
        trade_date = datetime.strptime(data['trade_date'], '%Y%m%d').strftime('%Y-%m-%d')
        
        if data['ts_code'] not in stock_info_by_ts_code:
            continue
            

        insert_sql = "INSERT INTO market_history (sid, open_price, highest_price,\
        lowest_price, closed_price, turnover_vol, trade_date)\
        VALUES (?, ?, ?, ?, ?, ?, ?)"

        sid = stock_info_by_ts_code[data['ts_code']]['sid']
        insert_data = (sid, data['open'], data['high'], data['low'], data['close'], data['vol'] ,trade_date)

        try:
            cur = conn.cursor()
            cur.execute(insert_sql, insert_data)
            conn.commit()
        except sqlite3.IntegrityError:
            pass
        except:
            print("Unexpected error when insert %s:",data.values, sys.exc_info())

    conn.close()

In [234]:
def get_market_trade_date_list(start_date):
    cal = pro.trade_cal(exchange='', start_date=start_date)

    trade_date = []
    for i in range(len(cal)):
        if cal.iloc[i]['is_open']:
            date = datetime.strptime(cal.iloc[i]['cal_date'], '%Y%m%d').strftime('%Y-%m-%d')
            trade_date.append(date)
            
    return trade_date

def get_not_record_date(sdate):
    
    compute_date_list = []
    trade_date = get_market_trade_date_list(sdate)

    conn = sqlite3.connect('../9900-test-db.sqlite')
    select_sql = "select trade_date from market_history group by trade_date"
    cur = conn.cursor()
    cur.execute(select_sql)
    data = [i[0] for i in cur]
    conn.close()

    start_date = datetime.strptime(sdate, '%Y%m%d')
    date = start_date
    while date < datetime.now():
        date_str = date.strftime('%Y-%m-%d')
        if date_str not in data and date_str in trade_date:
            compute_date_list.append(date)

        date += timedelta(days=1)

    return compute_date_list

In [236]:
start_date_str = "20191201"
date_list = get_not_record_date(start_date_str)
count = 0
total = len(date_list)
for date in date_list[0:1]:
    df = pro.daily(trade_date=date.strftime('%Y%m%d'))
    save_trade_df(df)
    count += 1
    time.sleep(0.15)
    print("finish save date = %s trade_date, processing(%s/%s)" % (date.strftime('%Y%m%d'), count, total))

finish save date = 20191203 trade_date, processing(1/21)


In [232]:
trade_date = get_market_trade_date_list(start_date_str)